In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls 

gdrive	sample_data


In [0]:

import os
os.chdir('gdrive/My Drive/kaggle/energy/zip_files')  #change dir


# import dataset from kaggle

In [0]:
# Colab library to upload files to notebook
from google.colab import files
# Install Kaggle library
!pip install -q kaggle
# Upload the kaggle.json file
uploaded = files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json
# Uploading the data 


In [0]:
!kaggle competitions download -c ashrae-energy-prediction -p /content/gdrive/My\ Drive/kaggle/energy

 98% 117M/120M [00:03<00:00, 32.2MB/s]
100% 120M/120M [00:03<00:00, 35.1MB/s]
 95% 158M/167M [00:02<00:00, 62.6MB/s]
100% 167M/167M [00:02<00:00, 65.2MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 5.20MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 76.9MB/s]
 89% 79.0M/88.4M [00:00<00:00, 97.0MB/s]
100% 88.4M/88.4M [00:00<00:00, 95.7MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 78.1MB/s]


# train.csv

In [0]:
import numpy as np
import pandas as pd

In [0]:
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
main = pd.read_csv ('train.csv.zip', compression='zip')
main.shape

(20216100, 4)

# building_metadata.csv

In [0]:
!ls

building_metadata.csv	   test.csv.zip       weather_test.csv.zip
model.h5		   train.csv.zip      weather_train.csv.zip
sample_submission.csv.zip  train_test.pickle


In [0]:
buildings = pd.read_csv ('building_metadata.csv' )

In [0]:
buildings.tail()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
1444,15,1444,Entertainment/public assembly,19619,1914.0,NaN
1445,15,1445,Education,4298,NaN,NaN
1446,15,1446,Entertainment/public assembly,11265,1997.0,NaN
1447,15,1447,Lodging/residential,29775,2001.0,NaN
1448,15,1448,Office,92271,2001.0,NaN


In [0]:
buildings.site_id.nunique()

16

In [0]:
buildings = buildings.drop(columns=['year_built','floor_count'], axis=1)


In [0]:
buildings.primary_use.value_counts()

Education                        549
Office                           279
Entertainment/public assembly    184
Public services                  156
Lodging/residential              147
Other                             25
Healthcare                        23
Parking                           22
Warehouse/storage                 13
Manufacturing/industrial          12
Retail                            11
Services                          10
Technology/science                 6
Food sales and service             5
Utility                            4
Religious worship                  3
Name: primary_use, dtype: int64

In [0]:
l=["Religious worship","Utility","Food sales and service","Technology/science","Services","Retail","Manufacturing/industrial","Warehouse/storage","Parking","Healthcare","Other"]

In [0]:
buildings.loc[buildings.primary_use.isin(l),'primary_use'] = 'others'

In [0]:
buildings["primary_use"] = buildings["primary_use"].astype('category')
buildings["primary_use"] = buildings["primary_use"].cat.codes

In [0]:
buildings['primary_use'] = buildings.primary_use.astype("int64")


In [0]:
buildings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 4 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null int64
square_feet    1449 non-null int64
dtypes: int64(4)
memory usage: 45.4 KB


# weather_train.csv

In [0]:
weather=pd.read_csv('weather_train.csv.zip', compression='zip')
weather.shape

(139773, 9)

In [0]:
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [0]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null object
air_temperature       139718 non-null float64
cloud_coverage        70600 non-null float64
dew_temperature       139660 non-null float64
precip_depth_1_hr     89484 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


In [0]:
weather.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [0]:
weather = weather.drop(columns=['precip_depth_1_hr','wind_direction','sea_level_pressure','cloud_coverage'], axis=1)


In [0]:
weather['wind_speed'].fillna(weather['wind_speed'].mean(), inplace=True)
weather['air_temperature'].fillna(weather['air_temperature'].mean(), inplace=True)
weather['dew_temperature'].fillna(weather['dew_temperature'].mean(), inplace=True)


# Merge

In [0]:
data =  pd.merge(main,buildings, on=['building_id'],sort=False)

In [0]:
data=pd.merge(data, weather, on=['site_id','timestamp'],sort=False)


In [0]:
data.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,0,7432,25.0,20.0,0.0
1,0,0,2016-01-01 01:00:00,0.0,0,0,7432,24.4,21.1,1.5
2,0,0,2016-01-01 02:00:00,0.0,0,0,7432,22.8,21.1,0.0
3,0,0,2016-01-01 03:00:00,0.0,0,0,7432,21.1,20.6,0.0
4,0,0,2016-01-01 04:00:00,0.0,0,0,7432,20.0,20.0,2.6


In [0]:
data["month"] = data["timestamp"].map(lambda row: row.split(" ")[0].split("-")[1])
data["day"] = data["timestamp"].map(lambda row: row.split(" ")[0].split("-")[2])
#data["time"] = data["timestamp"].map(lambda row: row.split(" ")[1].split(":")[0])


data['month'] = pd.to_numeric(data.month, errors='coerce')
#data['time'] = pd.to_numeric(data.time, errors='coerce')
data['day'] = pd.to_numeric(data.month, errors='coerce')


In [0]:

data = data.drop(columns=[ 'timestamp','site_id'], axis=1)


# Split Train_Test

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_df,test_df = train_test_split(data, test_size=0.2, random_state=45)


In [0]:
train_df.head()


,building_id,meter,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
10596157,922,0,148.000,0,147205,31.1,22.8,3.6,4,4
5498999,449,0,44.200,2,18340,7.8,6.7,4.6,12,12
4366511,318,0,245.790,3,105000,23.3,15.6,6.7,9,9
3482081,250,1,331.217,0,131710,25.6,10.0,2.6,10,10
1503341,144,0,56.625,3,57673,2.0,1.3,1.0,11,11


In [0]:
y_test = test_df['meter_reading']
x_test = test_df[['meter', 'building_id', 'primary_use', 'month', 'day','air_temperature','dew_temperature', 'wind_speed','square_feet']]

y_train = train_df['meter_reading']
x_train = train_df[['meter', 'building_id', 'primary_use', 'month', 'day','air_temperature','dew_temperature', 'wind_speed','square_feet']]


In [0]:
print(x_test.shape)
print(y_test.shape)
print("******************************")
print(x_train.shape)
print(y_train.shape)

(4025121, 9)
(4025121,)
******************************
(16100484, 9)
(16100484,)


In [0]:
x_train.head()

,meter,building_id,primary_use,month,day,air_temperature,dew_temperature,wind_speed,square_feet
9606115,0,835,1,12,12,13.9,11.1,4.1,17220
184779,1,78,0,3,3,22.2,16.1,4.1,76257
10846520,2,955,0,6,6,33.3,23.9,2.6,198488
14979167,1,1139,3,9,9,20.0,10.6,2.6,336650
1075753,1,97,2,12,12,17.8,6.7,0.0,187647


In [0]:
y_test.head()

13172218     102.2740
1594888      219.6000
1974942      212.4530
8695104     3682.6000
18226576      94.0657
Name: meter_reading, dtype: float64

In [0]:
# store the clean data in a pickle file to be used later in the modeling phase

import pickle
with open('train_test.pickle', 'wb') as f:
    pickle.dump([x_train, y_train, x_test,y_test], f)

# Run and evaluate Model

In [0]:
!ls

building_metadata.csv	   test.csv.zip       weather_test.csv.zip
model.h5		   train.csv.zip      weather_train.csv.zip
sample_submission.csv.zip  train_test.pickle


In [42]:
import pickle
with open('train_test.pickle', 'rb') as f:
    [x_train,y_train, x_test,y_test] = pickle.load(f)
x_train.shape,y_train.shape, x_test.shape,y_test.shape

((16100484, 9), (16100484,), (4025121, 9), (4025121,))

In [0]:
x_train.shape,y_train.shape, x_test.shape,y_test.shape

((16100484, 9), (16100484,), (4025121, 9), (4025121,))

In [11]:
from sklearn import preprocessing
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam
from keras import regularizers

Using TensorFlow backend.


In [0]:
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [13]:

model = Sequential()
model.add(Dense(500, input_dim=9, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(100, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(50, activation= "relu"))
model.add(Dense(1))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
#opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
#from keras.optimizers import Adam

def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))
  
metrics=root_mean_squared_error
model.compile(loss='mse', optimizer="rmsprop" ,metrics=[metrics])


In [0]:
y_train = np.log1p(y_train)


In [0]:
es = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True, mode='auto')

In [0]:
model.fit(x_train,y_train,epochs=30,batch_size=500,verbose=1,validation_data=(x_test,y_test),callbacks=[ es])





Train on 16100484 samples, validate on 4025121 samples
Epoch 1/30





16100484/16100484 [==============================] - 230s 14us/step - loss: 54590.7747 - root_mean_squared_error: 9.1164 - val_loss: 4.5202 - val_root_mean_squared_error: 2.1251
Epoch 2/30
16100484/16100484 [==============================] - 238s 15us/step - loss: 6.1387 - root_mean_squared_error: 2.1623 - val_loss: 4.3839 - val_root_mean_squared_error: 2.0927
Epoch 3/30
16100484/16100484 [==============================] - 241s 15us/step - loss: 4.9936 - root_mean_squared_error: 2.1086 - val_loss: 3.9984 - val_root_mean_squared_error: 1.9983
Epoch 4/30
16100484/16100484 [==============================] - 240s 15us/step - loss: 5.7887 - root_mean_squared_error: 2.0353 - val_loss: 3.7409 - val_root_mean_squared_error: 1.9330
Epoch 5/30
16100484/16100484 [==============================] - 237s 15us/step - loss: 3.8421 - root_mean_squared_error: 1.9389 - val_loss: 3.6891 - val_root_mean_squared_error: 1.9196
Epoch 6/

In [0]:
from keras.models import load_model
model.save('my_model2.h5')  # creates a HDF5 file 'my_model.h5'

In [15]:
model.load_weights("my_model2.h5")



In [28]:
y_test = np.log1p(y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log1p
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log1p
  """Entry point for launching an IPython kernel.


In [47]:
y_predicted= model.predict(x_test ,batch_size=1024, verbose=1)


4025121/4025121 [==============================] - 7s 2us/step


In [81]:
y_predicted[5555]

array([3.6932733], dtype=float32)

In [0]:
y_test = np.log1p(y_test)

In [0]:
from sklearn.metrics import mean_squared_error


In [49]:
print(np.sqrt(mean_squared_error(y_test,y_predicted)))


1.874762639177629


# data preparation for testset

In [4]:
!ls

building_metadata.csv  sample_submission.csv.zip  weather_test.csv.zip
model.h5	       test.csv.zip		  weather_train.csv.zip
my_model2.h5	       train.csv.zip
my_model.h5	       train_test.pickle


In [50]:
main_test = pd.read_csv ('test.csv.zip', compression='zip')
main_test.shape

(41697600, 4)

In [0]:
buildings_test = pd.read_csv ('building_metadata.csv' )

buildings_test = buildings_test.drop(columns=['year_built','floor_count'], axis=1)

l=["Religious worship","Utility","Food sales and service","Technology/science","Services","Retail","Manufacturing/industrial","Warehouse/storage","Parking","Healthcare","Other"]
buildings_test.loc[buildings_test.primary_use.isin(l),'primary_use'] = 'others'

buildings_test["primary_use"] = buildings_test["primary_use"].astype('category')
buildings_test["primary_use"] = buildings_test["primary_use"].cat.codes


In [0]:
weather_test=pd.read_csv('weather_test.csv.zip', compression='zip')


In [53]:
weather_test.shape

(277243, 9)

In [0]:
weather_test=pd.read_csv('weather_test.csv.zip', compression='zip')
weather_test.shape
weather_test = weather_test.drop(columns=['precip_depth_1_hr','wind_direction','sea_level_pressure','cloud_coverage'], axis=1)


weather_test['wind_speed'].fillna(weather_test['wind_speed'].mean(), inplace=True)
weather_test['air_temperature'].fillna(weather_test['air_temperature'].mean(), inplace=True)
weather_test['dew_temperature'].fillna(weather_test['dew_temperature'].mean(), inplace=True)



In [0]:
data_test =  pd.merge(main_test,buildings_test, on=['building_id'],sort=False)


In [0]:
data_test = data_test.merge(weather_test,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')

In [57]:
data_test.shape

(41697600, 10)

In [0]:
data_test["month"] = data_test["timestamp"].map(lambda row: row.split(" ")[0].split("-")[1])
data_test["day"] = data_test["timestamp"].map(lambda row: row.split(" ")[0].split("-")[2])
#data_test["time"] = data_test["timestamp"].map(lambda row: row.split(" ")[1].split(":")[0])

data_test['month'] = pd.to_numeric(data_test.month, errors='coerce')
#data_test['time'] = pd.to_numeric(data_test.time, errors='coerce')
data_test['day'] = pd.to_numeric(data_test.month, errors='coerce')

In [59]:
data_test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
0,0,0,0,2017-01-01 00:00:00,0,0,7432,17.8,11.7,3.6,1,1
1,129,0,0,2017-01-01 01:00:00,0,0,7432,17.8,12.8,3.1,1,1
2,258,0,0,2017-01-01 02:00:00,0,0,7432,16.1,12.8,3.1,1,1
3,387,0,0,2017-01-01 03:00:00,0,0,7432,17.2,13.3,3.1,1,1
4,516,0,0,2017-01-01 04:00:00,0,0,7432,16.7,13.3,2.6,1,1


In [0]:

data_test = data_test.drop(columns=[ 'row_id','timestamp','site_id'], axis=1)



In [62]:
data_test.shape

(41697600, 10)

In [97]:
predicted=model.predict(data_test, batch_size=1024, verbose=1)

41697600/41697600 [==============================] - 73s 2us/step


# submit

In [0]:
ids = data_test.row_id


(41697600, 2)

In [69]:
len(predicted)

41697600

In [104]:
predicted[5599995]

array([83.33016], dtype=float32)

In [0]:
submit = pd.read_csv('sample_submission.csv.zip', compression='zip')

# x_test = BTW_test[['meter', 'building_id', 'primary_use', 'Month', 'Day','air_temperature', 'wind_speed', 'precip_depth_1_hr', 'cloud_coverage',#
#        'square_feet']]
# x_test = x_test.values[:]
# x_test = x_test.reshape((x_test.shape[0],1,x_test.shape[-1]))

In [103]:
predicted = np.expm1(predicted)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in expm1
  """Entry point for launching an IPython kernel.


In [0]:
submit['meter_reading'] = predicted

In [105]:
submit.head()

,row_id,meter_reading
0,0,83.330162
1,1,83.330162
2,2,83.330162
3,3,83.330162
4,4,83.330162


In [109]:
submit.isnull().sum()

row_id           0
meter_reading    0
dtype: int64

In [115]:
print("a")

a


In [0]:
export_csv = submit.to_csv('./my-submission.csv', index=False)


In [73]:
submit.head()

,row_id,meter_reading
0,0,83.330162
1,1,83.330162
2,2,83.330162
3,3,83.330162
4,4,83.330162


In [0]:
83.330162

In [117]:
!kaggle competitions submit ashrae-energy-prediction -f my-submission.csv -m 'Final submission'


100% 570M/570M [00:12<00:00, 48.7MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III